In [2]:
#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
p =  configParser.get('extractFacesLatents', 'dbChunk')

# TODO Better display of progress and handling of exceptions
contLoop = True # Flag to continue to get chunks of videos from database
con = sl.connect(datasetPathDatabase)
offset = 0
while(contLoop):
    data = con.execute('''SELECT F.LATENT_REP,A.SPEAKER_EMB FROM VIDEO V 
                        INNER JOIN FACE F ON V.ID = F.VIDEO_ID 
                       INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID 
                       WHERE F.LATENT_REP IS NOT NULL AND AUDIO_PRE = 1 AND FACES_PRE = 1
                        ''')
    contLoop = False
    offset = offset + int(p)
    print("Got chunk of DATA from database. Training diffusion network")
    dataGotten = data.fetchall()
    for latent_rep_pickled, speaker_emb_pickled in dataGotten:
        latent_rep_list.append(pickle.loads(latent_rep_pickled))
        speaker_emb_list.append(pickle.loads(speaker_emb_pickled))


con.close()
        
        





Got chunk of DATA from database. Training diffusion network


In [ ]:
import torch
from imagen_pytorch import Unet, Imagen

# unet for imagen

unet1 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)


# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    unets = (unet1),
    image_sizes = (64, 64),
    timesteps = 1000,
    cond_drop_prob = 0.1
).cuda()

# mock images (get a lot of this) and text encodings from large T5

text_embeds = torch.randn(4, 256, 768).cuda()
images = torch.randn(4, 3, 256, 256).cuda()

# feed images into imagen, training each unet in the cascade

for i in (1):
    loss = imagen(images, text_embeds = text_embeds, unet_number = i)
    loss.backward()

# do the above for many many many many steps
# now you can sample an image based on the text embeddings from the cascading ddpm

images = imagen.sample(texts = [
    'a whale breaching from afar',
    'young girl blowing out candles on her birthday cake',
    'fireworks with blue and green sparkles'
], cond_scale = 3.)